In [1]:
# for preprocessing
import os
import numpy as np
import geopandas as gpd
from shapely.geometry import mapping
import rasterio
from rasterio.mask import mask


# for converting rasters into numerical data
import numpy as np
import pandas as pd
import rasterio
from rasterio.transform import xy
import statsmodels.api as sm

DATASETS 

* Y: tree cover loss 2000-2019 (30*30): https://earthenginepartners.appspot.com/science-2013-global-forest/download_v1.7.html

* X1: topography (30*30): https://portal.opentopography.org/raster?opentopoID=OTSDEM.092022.3035.1

* X2 CORINE Land Cover Classification (100*100): https://land.copernicus.eu/en/products/corine-land-cover

* X3 Population counts (100*100) https://hub.worldpop.org/geodata/listing?id=29





# **Y preprosessing**

In [ ]:
###########################################################################################################
# data from Global Forest Change 2000–2019: 
# https://earthenginepartners.appspot.com/science-2013-global-forest/download_v1.7.html
# To download individual 10x10 degree granules, click on a region on the map below and 
# then click on the URLs underneath it.
###########################################################################################################


# path to data
KOSOVO_BOUNDARY = r"C:/Users/oleks/Desktop/DEFORESTATION/3 REGRESSION/DATA/Y forest change/boundary/gadm41_XKO_0.shp"
base_dir = r"C:/Users/oleks/Desktop/DEFORESTATION/3 REGRESSION/DATA/Y forest change/hansen"
out_folder = r"C:/Users/oleks/Desktop/DEFORESTATION/3 REGRESSION/DATA/Y clipped"  # create a folder for an output

os.makedirs(out_folder, exist_ok=True)


# name of initially downloaded files that will be used in clipping
prefix = "Hansen_GFC-2024-v1.12"

rasters = {
    "treecover2000": os.path.join(base_dir, f"{prefix}_treecover2000_50N_020E.tif"),
    "lossyear":      os.path.join(base_dir, f"{prefix}_lossyear_50N_020E.tif"),
    "gain":          os.path.join(base_dir, f"{prefix}_gain_50N_020E.tif"),
    "datamask":      os.path.join(base_dir, f"{prefix}_datamask_50N_020E.tif"),
    "first":         os.path.join(base_dir, f"{prefix}_first_50N_020E.tif"),
    "last":          os.path.join(base_dir, f"{prefix}_last_50N_020E.tif"),
}

In [ ]:
# data Kosovo shapefile (The boundaries of Kosovo) from:
# https://www.eea.europa.eu/data-and-maps/data/eea-reference-grids-2/gis-files/kosovo-shapefile

# Load Kosovo boundary
kosovo = gpd.read_file(KOSOVO_BOUNDARY)

# Open one raster to get its CRS
with rasterio.open(rasters["treecover2000"]) as src:
    raster_crs = src.crs

# Reproject Kosovo boundary to raster CRS
kosovo = kosovo.to_crs(raster_crs)

# Convert geometries to GeoJSON-like dicts for rasterio.mask
mask_geoms = [mapping(geom) for geom in kosovo.geometry]

In [11]:
import os

def clip_raster(in_path, out_path=None):
    """
    Clip a raster to Kosovo using the previously defined mask_geoms.
    Returns (array, transform, profile).
    """
    with rasterio.open(in_path) as src:
        out_image, out_transform = mask(src, mask_geoms, crop=True)
        out_meta = src.meta.copy()

    # Update metadata
    out_meta.update({
        "height": out_image.shape[1],
        "width":  out_image.shape[2],
        "transform": out_transform
    })

    if out_path is not None:
        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        with rasterio.open(out_path, "w", **out_meta) as dst:
            dst.write(out_image)

    return out_image, out_transform, out_meta

In [ ]:
clipped = {}

for name, path in rasters.items():
    out_path = f"{out_folder}/{name}_Kosovo.tif"
    arr, transform, meta = clip_raster(path, out_path=out_path)
    clipped[name] = {"array": arr, "transform": transform, "meta": meta}

# **X1 preprosessing**

In [15]:
# -------------------------------------------------------------------
# 1. USER INPUT: set your file paths here
# -------------------------------------------------------------------
KOSOVO_BOUNDARY = r"C:/Users/oleks/Desktop/DEFORESTATION/3 REGRESSION/DATA/Y forest change/boundary/gadm41_XKO_0.shp"
TOPO_RASTER     = r"C:/Users/oleks/Desktop/DEFORESTATION/3 REGRESSION/DATA/X1 topography/output_be.tif"
OUTPUT_RASTER   = r"C:/Users/oleks/Desktop/DEFORESTATION/3 REGRESSION/DATA/X1 clipped/output_be_Kosovo.tif"

# Make sure output directory exists
os.makedirs(os.path.dirname(OUTPUT_RASTER), exist_ok=True)

# -------------------------------------------------------------------
# 2. Load Kosovo boundary
# -------------------------------------------------------------------
kosovo = gpd.read_file(KOSOVO_BOUNDARY)

if kosovo.empty:
    raise ValueError("Kosovo boundary file has no features. Check the shapefile/path.")

print("Boundary CRS:", kosovo.crs)

# -------------------------------------------------------------------
# 3. Open topography raster and reproject boundary if needed
# -------------------------------------------------------------------
with rasterio.open(TOPO_RASTER) as src:
    print("Topography CRS:", src.crs)

    # Reproject boundary to raster CRS if they differ
    if kosovo.crs is None:
        raise ValueError("Kosovo boundary CRS is undefined. Set it to EPSG:4326 in QGIS, then try again.")

    if kosovo.crs != src.crs:
        kosovo = kosovo.to_crs(src.crs)
        print("Reprojected boundary to raster CRS:", kosovo.crs)

    # Prepare geometries for rasterio.mask
    mask_geoms = [mapping(geom) for geom in kosovo.geometry]

    # ----------------------------------------------------------------
    # 4. Clip raster by Kosovo boundary
    # ----------------------------------------------------------------
    out_image, out_transform = mask(
        src,
        mask_geoms,
        crop=True  # crop to boundary extent
    )

    # Copy and update metadata
    out_meta = src.meta.copy()
    out_meta.update({
        "height": out_image.shape[1],
        "width":  out_image.shape[2],
        "transform": out_transform
    })

# -------------------------------------------------------------------
# 5. Save clipped raster
# -------------------------------------------------------------------
with rasterio.open(OUTPUT_RASTER, "w", **out_meta) as dst:
    dst.write(out_image)

print("✅ Saved clipped topography raster to:")
print("   ", OUTPUT_RASTER)

# -------------------------------------------------------------------
# 6. (Optional) If you want the array in Python afterwards:
# -------------------------------------------------------------------
# topo_array = out_image   # shape: (bands, rows, cols)
# print("Array shape:", topo_array.shape)

Boundary CRS: EPSG:4326
Topography CRS: EPSG:4326
✅ Saved clipped topography raster to:
    C:/Users/oleks/Desktop/DEFORESTATION/3 REGRESSION/DATA/X1 clipped/output_be_Kosovo.tif


# **CONVERTING RASTERS INTO DATA**

In [1]:
import numpy as np
import pandas as pd
import rasterio
from rasterio.transform import xy
from rasterio.warp import reproject, Resampling
from sklearn.linear_model import LogisticRegression

# ------------------------------------------------------------------
# paths
# ------------------------------------------------------------------
dem_path       = r"C:/Users/oleks/Desktop/DEFORESTATION/3 REGRESSION/DATA/X1 clipped/topography_be_Kosovo.tif"
lossyear_path  = r"C:/Users/oleks/Desktop/DEFORESTATION/3 REGRESSION/DATA/Y clipped/lossyear_Kosovo.tif"
datamask_path  = r"C:/Users/oleks/Desktop/DEFORESTATION/3 REGRESSION/DATA/Y clipped/datamask_Kosovo.tif"

def compute_slope(dem, transform, nodata=None):
    dem = dem.astype("float64")
    if nodata is not None:
        dem = np.where(dem == nodata, np.nan, dem)

    # pixel size in x,y *in DEM units* (should be meters)
    xres = transform.a
    yres = -transform.e

    dZ_dy, dZ_dx = np.gradient(dem, yres, xres)
    slope_rad = np.arctan(np.sqrt(dZ_dx**2 + dZ_dy**2))
    slope_deg = np.degrees(slope_rad)
    return slope_deg

# ------------------------------------------------------------------
# 1. Read Hansen rasters (reference grid)
# ------------------------------------------------------------------
with rasterio.open(lossyear_path) as src_loss:
    lossyear = src_loss.read(1)
    lossyear_nodata = src_loss.nodata
    ref_transform = src_loss.transform
    ref_crs = src_loss.crs
    height, width = src_loss.shape

with rasterio.open(datamask_path) as src_mask:
    datamask = src_mask.read(1)
    datamask_nodata = src_mask.nodata

print("lossyear shape:", lossyear.shape)

# ------------------------------------------------------------------
# 2. Read DEM and compute slope in DEM CRS (meters)
# ------------------------------------------------------------------
with rasterio.open(dem_path) as src_dem:
    dem_src = src_dem.read(1)
    dem_src_nodata = src_dem.nodata
    dem_src_transform = src_dem.transform
    dem_src_crs = src_dem.crs

slope_src = compute_slope(dem_src, dem_src_transform, nodata=dem_src_nodata)

# quick sanity check on slope range
print("Slope_src min/max:", np.nanmin(slope_src), np.nanmax(slope_src))

# ------------------------------------------------------------------
# 3. Reproject DEM and slope to Hansen grid
# ------------------------------------------------------------------
dem = np.full((height, width), np.nan, dtype="float32")
slope = np.full((height, width), np.nan, dtype="float32")

with rasterio.open(dem_path) as src_dem:
    reproject(
        source=rasterio.band(src_dem, 1),
        destination=dem,
        src_transform=dem_src_transform,
        src_crs=dem_src_crs,
        dst_transform=ref_transform,
        dst_crs=ref_crs,
        resampling=Resampling.bilinear,
        src_nodata=dem_src_nodata,
        dst_nodata=np.nan,
    )

reproject(
    source=slope_src,
    destination=slope,
    src_transform=dem_src_transform,
    src_crs=dem_src_crs,
    dst_transform=ref_transform,
    dst_crs=ref_crs,
    resampling=Resampling.bilinear,
    dst_nodata=np.nan,
)

print("DEM (resampled) shape:", dem.shape)
print("Slope (resampled) min/max:", np.nanmin(slope), np.nanmax(slope))

# ------------------------------------------------------------------
# 4. Build Y and DataFrame (same as before)
# ------------------------------------------------------------------
Y = (lossyear > 0).astype(np.uint8)
forest_2000 = (datamask == 1)
Y = np.where(forest_2000, Y, np.nan)

if lossyear_nodata is not None:
    Y = np.where(lossyear == lossyear_nodata, np.nan, Y)

valid = np.isfinite(Y) & np.isfinite(dem) & np.isfinite(slope)
print("Valid pixels:", np.sum(valid))

max_samples = 200000
valid_indices = np.where(valid)
n_valid = valid_indices[0].size

if n_valid > max_samples:
    rng = np.random.default_rng(0)
    chosen = rng.choice(n_valid, size=max_samples, replace=False)
    rows = valid_indices[0][chosen]
    cols = valid_indices[1][chosen]
else:
    rows = valid_indices[0]
    cols = valid_indices[1]

Y_flat     = Y[rows, cols].astype(int)
elev_flat  = dem[rows, cols].astype(float)
slope_flat = slope[rows, cols].astype(float)

xs, ys = xy(ref_transform, rows, cols)
xs = np.array(xs)
ys = np.array(ys)

df = pd.DataFrame({
    "Y_deforestation": Y_flat,
    "elevation_m": elev_flat,
    "slope_deg": slope_flat,
    "x": xs,
    "y": ys
})

df.to_csv("kosovo_deforestation_elev_slope_fixed.csv", index=False)
print(df.head())

lossyear shape: (5592, 7173)
Slope_src min/max: 0.0 89.99987884607678
DEM (resampled) shape: (5592, 7173)
Slope (resampled) min/max: 0.0 89.999855
Valid pixels: 18920761
   Y_deforestation  elevation_m  slope_deg          x          y
0                0  2154.837402  89.998032  21.038125  42.165375
1                0   417.952515  89.989296  20.483625  42.666375
2                0   573.099976  89.986877  20.744875  42.523625
3                0   445.982513  89.979378  20.482875  42.743875
4                0  1045.612549  89.994110  20.214875  42.545875


# **RUN LOGISTIC REGRESSION**

In [2]:
data = pd.read_csv("kosovo_deforestation_elev_slope_fixed.csv")

Y = data["Y_deforestation"].values
X = data[["elevation_m", "slope_deg"]].values

log_reg = LogisticRegression(
    penalty=None,
    solver="lbfgs",
    max_iter=10000,
    tol=1e-15,
    fit_intercept=True,
).fit(X, Y)

print("Intercept:", log_reg.intercept_[0])
print("Coefficients:", log_reg.coef_.ravel())

Intercept: -10.577590162352527
Coefficients: [0.00056013 0.06815061]
